<a href="https://colab.research.google.com/github/Praveenlingala378/Armstrong/blob/main/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os


In [4]:
# Path to the dataset directory
dataset_dir = '/content/drive/MyDrive/dataset/'

# Define data generators for training and validation with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 172 images belonging to 3 classes.
Found 42 images belonging to 3 classes.


In [5]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 128)       0

In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/10
5/5 [==============================] - 32s 6s/step - loss: 1.7347 - accuracy: 0.4313 - val_loss: 1.0172 - val_accuracy: 0.6250
Epoch 2/10
5/5 [==============================] - 3s 733ms/step - loss: 1.0246 - accuracy: 0.5437 - val_loss: 0.9848 - val_accuracy: 0.5938
Epoch 3/10
5/5 [==============================] - 3s 546ms/step - loss: 1.0109 - accuracy: 0.5429 - val_loss: 1.0484 - val_accuracy: 0.5312
Epoch 4/10
5/5 [==============================] - 3s 736ms/step - loss: 1.0175 - accuracy: 0.5214 - val_loss: 1.0000 - val_accuracy: 0.5625
Epoch 5/10
5/5 [==============================] - 3s 723ms/step - loss: 0.9851 - accuracy: 0.5437 - val_loss: 0.9785 - val_accuracy: 0.5625
Epoch 6/10
5/5 [==============================] - 3s 615ms/step - loss: 0.9787 - accuracy: 0.5429 - val_loss: 1.0253 - val_accuracy: 0.4688
Epoch 7/10
5/5 [==============================] - 2s 485ms/step - loss: 0.9943 - accuracy: 0.5929 - val_loss: 1.0015 - val_accuracy: 0.5625
Epoch 8/10
5/5 [======

In [7]:
# Save the trained model
model.save('facial_recognition_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
def predict_person(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Load the saved model
    model = tf.keras.models.load_model('/content/facial_recognition_model.h5')

    prediction = model.predict(img_array)
    person_index = np.argmax(prediction)
    person_name = list(train_generator.class_indices.keys())[person_index]

    return person_name

# Example usage:
test_image_path = '/content/Akshay Kumar_3.jpg'
predicted_person = predict_person(test_image_path)
print(f"Predicted person: {predicted_person}")


1/1 [==============================] - 0s 373ms/step
Predicted person: Vijay Deverakonda
